In [36]:
import os
import re  
import PIL.Image as Image
from PIL import ImageDraw,ImageFont,ImageOps


def resize_by_width(infile, image_size):
    im = Image.open(infile)
    (x, y) = im.size
    lv = round(x / image_size, 2) + 0.01
    x_s = int(x // lv)
    y_s = int(y // lv)
    print("x_s", x_s, y_s)
    out = im.resize((x_s, y_s), Image.LANCZOS)
    blank_image = Image.new("RGB", (x, y), (0, 0, 0))
    blank_image.paste(out, (0, 0))
    pad_height = 100  # 需要调整  块的高度
    pad_width = x  
    pad_color = (0, 0, 0)  
    out = ImageOps.pad(blank_image, (pad_width, pad_height + y), method=Image.Resampling.NEAREST, color=pad_color)
    
    return out




def get_new_img_xy(infile, image_size):
    im = Image.open(infile)
    (x, y) = im.size
    y+=200  # 需要调整，炼丹
    lv = round(x / image_size, 2) + 0.01
    x_s = x // lv
    y_s = y // lv
    # print("x_s", x_s, y_s)
    # out = im.resize((x_s, y_s), Image.ANTIALIAS)
    return x_s, y_s


# 定义图像拼接函数
def image_compose(image_colnum, image_size, image_rownum, image_names, image_save_path, x_new, y_new):
    print(image_names)
    to_image = Image.new('RGB', (image_colnum * x_new, image_rownum * y_new))  # 创建一个新图
    draw = ImageDraw.Draw(to_image)
    fnt = ImageFont.truetype(r'C:\Windows\Fonts\simkai.ttf', 50) 
    total_num = 0
    for y in range(1, image_rownum + 1):
        for x in range(1, image_colnum + 1):
            from_image = resize_by_width(image_names[image_colnum * (y - 1) + x - 1], image_size)
            to_image.paste(from_image, ((x - 1) * x_new, (y - 1) * y_new))
            print(total_num)
            total_num += 1
            title = image_names[total_num-1].split('\\')[-1]  
            print(title)
            first_underscore_index = title.index("_")  
            last_underscore_index = title.rindex("_")  
            title = title[first_underscore_index + 1:last_underscore_index] 
            print(title)
            draw.text( ((x - 1) * x_new, (y - 1) * y_new), title, fill="white",font=fnt)        
            if total_num == len(image_names):
                break
    return to_image.save(image_save_path)  # 保存新图



def get_image_list_fullpath(dir_path):
    file_name_list = os.listdir(dir_path)
    image_fullpath_list = []
    for file_name_one in file_name_list:
        file_one_path = os.path.join(dir_path, file_name_one)
        if os.path.isfile(file_one_path):
            image_fullpath_list.append(file_one_path)
        else:
            img_path_list = get_image_list_fullpath(file_one_path)
            image_fullpath_list.extend(img_path_list)
    return image_fullpath_list


def merge_images(image_dir_path,image_size,image_colnum):
    image_fullpath_list = get_image_list_fullpath(image_dir_path)
    print("image_fullpath_list", len(image_fullpath_list), image_fullpath_list)

    image_save_path = r'{}.jpg'.format(image_dir_path)  # 图片转换后的地址
    image_rownum_yu = len(image_fullpath_list) % image_colnum
    if image_rownum_yu == 0:
        image_rownum = len(image_fullpath_list) // image_colnum
    else:
        image_rownum = len(image_fullpath_list) // image_colnum + 1

    x_list = []
    y_list = []
    for img_file in image_fullpath_list:
        img_x, img_y = get_new_img_xy(img_file, image_size)
        x_list.append(img_x)
        y_list.append(img_y)

    print("x_list", sorted(x_list))
    print("y_list", sorted(y_list))
    x_new = int(x_list[len(x_list) // 5 * 4])
    y_new = int(y_list[len(y_list) // 5 * 4])
    print(" x_new, y_new", x_new, y_new)
    image_compose(image_colnum, image_size, image_rownum, image_fullpath_list, image_save_path, x_new, y_new)  

In [37]:
image_dir_path = r'C:\Users\Jun Rao\Desktop\综合办公\img_merge\血红蛋白-75mg'  # 图片集地址
image_size = 1080  # 每张小图片的大小
image_colnum = 4  # 合并成一张图后，一行有几个小图
merge_images(image_dir_path, image_size, image_colnum)

image_fullpath_list 8 ['C:\\Users\\Jun Rao\\Desktop\\综合办公\\img_merge\\血红蛋白-75mg\\Project_X-2.5-1-400_ch00.tif', 'C:\\Users\\Jun Rao\\Desktop\\综合办公\\img_merge\\血红蛋白-75mg\\Project_X-2.5-1-800_ch00.tif', 'C:\\Users\\Jun Rao\\Desktop\\综合办公\\img_merge\\血红蛋白-75mg\\Project_X-2.5-2-400_ch00.tif', 'C:\\Users\\Jun Rao\\Desktop\\综合办公\\img_merge\\血红蛋白-75mg\\Project_X-2.5-2-800_ch00.tif', 'C:\\Users\\Jun Rao\\Desktop\\综合办公\\img_merge\\血红蛋白-75mg\\Project_X-5.5-1-400_ch00.tif', 'C:\\Users\\Jun Rao\\Desktop\\综合办公\\img_merge\\血红蛋白-75mg\\Project_X-5.5-1-800_ch00.tif', 'C:\\Users\\Jun Rao\\Desktop\\综合办公\\img_merge\\血红蛋白-75mg\\Project_X-5.5-2-400_ch00.tif', 'C:\\Users\\Jun Rao\\Desktop\\综合办公\\img_merge\\血红蛋白-75mg\\Project_X-5.5-2-800_ch00.tif']
x_list [1072.0, 1072.0, 1072.0, 1072.0, 1072.0, 1072.0, 1072.0, 1072.0]
y_list [916.0, 916.0, 916.0, 916.0, 916.0, 916.0, 916.0, 916.0]
 x_new, y_new 1072 916
['C:\\Users\\Jun Rao\\Desktop\\综合办公\\img_merge\\血红蛋白-75mg\\Project_X-2.5-1-400_ch00.tif', 'C:\\Users\\Jun 

In [1]:
import cv2

ModuleNotFoundError: No module named 'cv2'